In [56]:
import os
import pandas as pd
from sqlalchemy import create_engine
import pycep_correios
from geopy.geocoders import Nominatim, HereV7, TomTom
from geopy.extra.rate_limiter import RateLimiter

from dotenv import load_dotenv
load_dotenv()  # take environment variables from .env.
# conexão com o banco de dados SQL Server
engine = create_engine(
  os.environ.get('DATABASE_URL_SQL_SERVER'),
  connect_args = {"TrustServerCertificate": "yes"},
  echo=False
)

# conexão postgres
engine_postgres = create_engine(
  os.environ.get('DATABASE_URL_SQL_POSTGRES'),
  echo=False
)

# conexão mariadb/mysql
engine_mysql = create_engine(
  os.environ.get('DATABASE_URL_SQL_MYSQL'),
  echo=False
)

file_path = os.path.join('postos_saude_brasil.csv')
df = pd.read_csv(file_path)
df.head()

,UF,MUNICIPIO,ESTABELECIMENTO,LOGRADOURO,NUMERO,BAIRRO,ENDERECO_COMPLETO,latitude,longitude,address,point
0,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,RUA 07 QUEDAS,284,CENTRO,"RUA 07 QUEDAS, CENTRO, ACRELANDIA, AC, BRASIL",-10.07561,-67.05003,"Rua Sete Quedas, Acrelândia","10 4m 32.196s S, 67 3m 0.108s W"
1,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,BR 364 KM 114,114,ZONA RURAL,"BR 364 KM 114, ZONA RURAL, ACRELANDIA, AC, BRASIL",-22.28388,-51.91008,"Via de Acesso à Zona Rural, 114, 19260-000, Mi...","22 17m 1.968s S, 51 54m 36.288s W"
2,AC,ACRELANDIA,UNIDADE MISTA DE SAUDE DE ACRELANDIA,AVENIDA PARANA,346,CENTRO,"AVENIDA PARANA, CENTRO, ACRELANDIA, AC, BRASIL",-10.07684,-67.05564,"Avenida Paraná, 69945, Acrelândia","10 4m 36.624s S, 67 3m 20.304s W"
3,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,BR 364 KM 90 RAMAL GRANADA KM 16,16,ZONA RURAL,"BR 364 KM 90 RAMAL GRANADA KM 16, ZONA RURAL, ...",-17.34786,-49.93093,"Via de Acesso à Zona Rural, 90, 75940-000, Edéia","17 20m 52.296s S, 49 55m 51.348s W"
4,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA REDENCAO,AC 475 VILA REDENCAOI,NaN,ZONA RURAL,"AC 475 VILA REDENCAOI, ZONA RURAL, ACRELANDIA,...",-22.28064,-51.91064,"Via de Acesso à Zona Rural, 475, 19260-000, Mi...","22 16m 50.304s S, 51 54m 38.304s W"


In [57]:
# cria o campo endereço completo para facilitar a chamada à API
#df['ENDERECO_COMPLETO'] = df['LOGRADOURO'] + ', ' + df['BAIRRO'] + ', ' + df['MUNICIPIO'] + ', ' + df['UF'] + ', ' + 'BRASIL'
# inclui novas colunas
#df['latitude'] = None
#df['longitude'] = None
#df['address'] = None
#df['point'] = None
df.head()

,UF,MUNICIPIO,ESTABELECIMENTO,LOGRADOURO,NUMERO,BAIRRO,ENDERECO_COMPLETO,latitude,longitude,address,point
0,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA RICARDO MONTEIRO ROLA,RUA 07 QUEDAS,284,CENTRO,"RUA 07 QUEDAS, CENTRO, ACRELANDIA, AC, BRASIL",-10.07561,-67.05003,"Rua Sete Quedas, Acrelândia","10 4m 32.196s S, 67 3m 0.108s W"
1,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA JOAO DANIEL DAMASCENO,BR 364 KM 114,114,ZONA RURAL,"BR 364 KM 114, ZONA RURAL, ACRELANDIA, AC, BRASIL",-22.28388,-51.91008,"Via de Acesso à Zona Rural, 114, 19260-000, Mi...","22 17m 1.968s S, 51 54m 36.288s W"
2,AC,ACRELANDIA,UNIDADE MISTA DE SAUDE DE ACRELANDIA,AVENIDA PARANA,346,CENTRO,"AVENIDA PARANA, CENTRO, ACRELANDIA, AC, BRASIL",-10.07684,-67.05564,"Avenida Paraná, 69945, Acrelândia","10 4m 36.624s S, 67 3m 20.304s W"
3,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA CICERO BATISTA,BR 364 KM 90 RAMAL GRANADA KM 16,16,ZONA RURAL,"BR 364 KM 90 RAMAL GRANADA KM 16, ZONA RURAL, ...",-17.34786,-49.93093,"Via de Acesso à Zona Rural, 90, 75940-000, Edéia","17 20m 52.296s S, 49 55m 51.348s W"
4,AC,ACRELANDIA,UNIDADE DE SAUDE DA FAMILIA REDENCAO,AC 475 VILA REDENCAOI,NaN,ZONA RURAL,"AC 475 VILA REDENCAOI, ZONA RURAL, ACRELANDIA,...",-22.28064,-51.91064,"Via de Acesso à Zona Rural, 475, 19260-000, Mi...","22 16m 50.304s S, 51 54m 38.304s W"


In [58]:
geolocator_nominatim = Nominatim(user_agent="info_vacinas")
geolocator_tomtom = TomTom(api_key='wvuAFmBut64DLjQqey19XtfuMXZilzbj', user_agent="info_vacinas")
geolocator_here = HereV7(apikey=os.environ.get('HERE_API_KEY'), user_agent="info_vacinas")
geocode_nominatim = RateLimiter(geolocator_nominatim.geocode, min_delay_seconds=0)
geocode_tomtom = RateLimiter(geolocator_tomtom.geocode, min_delay_seconds=0)
geocode_here = RateLimiter(geolocator_here.geocode, min_delay_seconds=0)
tamanho = len(df)

print(f'Processando {str(tamanho).zfill(5)} registros')
for index, row in reversed(df.iterrows()):
  # pula os registros que já possuem dados de localização
  if not row['latitude']:
    continue
  
  # inicializa a variável que receberá o retorno das APIs de localização
  location = None

  try:
    # tenta fazer a localização usando o Nominatim
    location = geolocator_nominatim.geocode(row['ENDERECO_COMPLETO'])
    print(location)
  except Exception as e:
    print(f'Erro: {e}')
    # se a localização não retorna nada, tenta pegar a localização no TomTom
    try:
      location = geolocator_tomtom.geocode(row['ENDERECO_COMPLETO'])
      print(location)
    except Exception as e:
      # se a localização não retorna nada, tenta pegar a localização no Here
      print(f'Erro: {e}')
      try:
        location = geolocator_here.geocode(row['ENDERECO_COMPLETO'])
        print(location)
      except Exception as e:
        # salva o CSV com os dados buscados até aquele momento
        df.to_csv(os.path.join('postos_saude_brasil.csv'), index=False)
        if not location:
          continue

  # se em nenhuma das três tentativas conseguiu retornar a localização, 
  # vai para o próximo registro
  if not location:
    continue

  # exibe apenas de 500 em 500 para saber como está o processo
  if index%100 == 0:
    print(f'Processando {str(index+1).zfill(5)} de {str(tamanho).zfill(5)} registros')
    df.to_csv(os.path.join('postos_saude_brasil.csv'), index=False)

  row['latitude'] = location.latitude
  row['longitude'] = location.longitude
  row['address'] = location.address
  row['point'] = location.point

df.to_csv(os.path.join('postos_saude_brasil.csv'), index=False)
df.head()

Processando 36360 registros


TypeError: 'generator' object is not reversible

In [ ]:
df.to_csv(os.path.join('postos_saude_brasil.csv'), index=False)